<a href="https://colab.research.google.com/github/zodbot/llm_finetuning/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Fine-tuned GPT-2 Sentiment Classifier

This notebook:
1. Loads previously fine-tuned GPT-2 model
2. Evaluates performance on test set
3. Provides function for classifying new reviews
4. Demonstrates usage with example reviews

Model details:
- Base: GPT-2 fine-tuned on SST-2
- Task: Binary sentiment classification
- Output: Positive/Negative with confidence scores

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import urllib.request
url = (
    "https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch05/"
    "01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

('gpt_download.py', <http.client.HTTPMessage at 0x7f8f7d936d90>)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

from gpt_download import download_and_load_gpt2
settings, params = download_and_load_gpt2(
    model_size="124M",
    models_dir="/content/drive/MyDrive/gpt2"
)

Mounted at /content/drive
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/checkpoint
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/encoder.json
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/hparams.json
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/model.ckpt.index
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: /content/drive/MyDrive/gpt2/124M/vocab.bpe


KeyboardInterrupt: 

In [ ]:
import sys


!git clone https://github.com/zodbot/llm_finetuning.git

# Change into repo directory
%cd llm_finetuning

from src.model import GPTModel
from src.config import GPT_CONFIGS
from src.utils import load_weights_into_gpt
import torch

# Get configuration
config = GPT_CONFIGS["gpt2-small (124M)"]

# Set up model for classification
model = GPTModel(config)
load_weights_into_gpt(model, params)
model.eval()

# Freeze model
for param in model.parameters():
    param.requires_grad = False

# Modify for classification
num_classes = 2
model.out_head = torch.nn.Linear(config["emb_dim"], num_classes)
# model.out_head = torch.nn.Sequential(
#     torch.nn.Dropout(0.3),
#     torch.nn.Linear(config["emb_dim"], num_classes)
# )
# Unfreeze specific layers
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True
for param in model.final_norm.parameters():
    param.requires_grad = True

In [ ]:
!pip install datasets

In [ ]:
# SST-2 (Stanford Sentiment Treebank) is a great dataset for binary sentiment classification
from datasets import load_dataset
ds = load_dataset('nyu-mll/glue', 'sst2', split='train')

In [ ]:
!pip install tiktoken

In [ ]:
# Convert to pandas DataFrame
import pandas as pd
import tiktoken

df = pd.DataFrame({
    'Label': ds['label'],
    'Text': ds['sentence']
})
tokenizer = tiktoken.get_encoding("gpt2")
print(len(df))
print(df["Label"].value_counts())

# Take random sample of 1000 rows
df_sample = df.sample(n=5000, random_state=42)  # random_state for reproducibility

# Optional: Look at the distribution of labels to ensure it's balanced
print("Label distribution in sample:")
print(df_sample['Label'].value_counts())

def random_split(df, train_frac, validation_frac):
    df = df.sample(
        frac=1, random_state=123
    ).reset_index(drop=True)
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(df, 0.7, 0.1)

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

f = open("train.csv")
inputs = []
max_length = 0
for line in f.read():
  ids = tokenizer.encode(line)
  max_length = max(max_length, len(ids))
  inputs.append(ids)


for input in inputs:
  for _ in range(max_length - len(input)):
    input.append(5027)



In [ ]:
import torch
from torch.utils.data import Dataset
# it identifies the longest sequence in the training dataset, encodes the text messages,
# and ensures that all other sequences are padded with a padding token to match the length of the longest sequence.
class Sst2DataSet(Dataset):
  def __init__(self, csv_file, tokenizer, max_length=None,
                 pad_token_id=50256):
      self.data = pd.read_csv(csv_file)
      self.encoded_texts = [tokenizer.encode(data) for data in self.data["Text"]]
      if max_length is None:
          self.max_length = self._longest_length()
      else:
          self.max_length = max_length
      # Truncates sequences if they are longer than max_length
      self.encoded_texts = [
                  encoded_text[:self.max_length]
                  for encoded_text in self.encoded_texts
      ]
      # add padding
      self.encoded_texts = [
        encoded_text + [pad_token_id] *
        (self.max_length - len(encoded_text))
        for encoded_text in self.encoded_texts
      ]


  def __getitem__(self, index):
      encoded = self.encoded_texts[index]
      label = self.data.iloc[index]["Label"]
      return (
          torch.tensor(encoded, dtype=torch.long),
          torch.tensor(label, dtype=torch.long)
      )
  def __len__(self):
      return len(self.data)

  def _longest_length(self):
      max_length = 0
      for encoded_text in self.encoded_texts:
          encoded_length = len(encoded_text)
          if encoded_length > max_length:
              max_length = encoded_length
      return max_length


train_dataset = Sst2DataSet(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

val_dataset = Sst2DataSet(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
test_dataset = Sst2DataSet(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
print(train_dataset.max_length)

In [ ]:
import torch


from torch.utils.data import DataLoader
num_workers = 0
batch_size = 8
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)
print("# of batches: ", len(train_loader))
for train, target in train_loader:
  print(train.shape, target.shape)
  break


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def calc_loss_batch(input_batch, target_batch, model, device):
  input_batch = input_batch.to(device)
  target_batch = target_batch.to(device)
  logits = model(input_batch)[:, -1, :]
  loss = torch.nn.functional.cross_entropy(logits, target_batch)
  return loss

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
      train_loss = calc_loss(
            train_loader, model, device, num_batches=eval_iter
        )
      val_loss = calc_loss(
            val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def train_classifier(model, data_loader, val_loader, optimizer, device,
               epoch_num, eval_freq, eval_iter):
  model.train()
  examples_seen = 0
  steps = -1
  train_losses, val_losses, train_accs, val_accs = [], [], [], []

  for epoch in range(epoch_num):
    model.train()
    for i, (input_batch, target_batch) in enumerate(data_loader):
      input_batch = input_batch.to(device)
      target_batch = target_batch.to(device)
      # reset optimizer
      optimizer.zero_grad()

      loss = calc_loss_batch(input_batch, target_batch, model, device)

      # cal loss gradient
      loss.backward()

      # update weight
      optimizer.step()
      # we care about number of smples not tokens here
      examples_seen += input_batch.shape[0]
      steps += 1

      if steps % eval_freq == 0:
          train_loss, val_loss = evaluate_model(
            model, data_loader, val_loader, device, eval_iter)
          train_losses.append(train_loss)
          val_losses.append(val_loss)
          print(f"Ep {epoch+1} (Step {steps:06d}): "
                f"Train loss {train_loss:.3f}, "
                f"Val loss {val_loss:.3f}")

    train_accuracy = calc_accuracy(
      data_loader, model, device, num_batches=eval_iter
    )
    val_accuracy = calc_accuracy(
      val_loader, model, device, num_batches=eval_iter
    )
    print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
    print(f"Validation accuracy: {val_accuracy*100:.2f}%")
    train_accs.append(train_accuracy)
    val_accs.append(val_accuracy)

  return train_losses, val_losses, train_accs, val_accs, examples_seen


In [ ]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.1)
num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = \
    train_classifier(
        model, train_loader, val_loader, optimizer, device,
        num_epochs, eval_freq=2000,
        eval_iter=5
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
# changed the weight decay
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = \
    train_classifier(
        model, train_loader, val_loader, optimizer, device,
        num_epochs, eval_freq=2000,
        eval_iter=5
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")



In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': config,
    'unfrozen_layers': ['trf_blocks[-1]', 'final_norm']  # document which layers were unfrozen
}, '/content/drive/MyDrive/gpt2_finetuned_sst.pt')

In [ ]:
import matplotlib.pyplot as plt

def plot_values(
        epochs_seen, examples_seen, train_values, val_values,
  label="loss"):
  fig, ax1 = plt.subplots(figsize=(5, 3))

  ax1.plot(epochs_seen, train_values, label=f"Training {label}")
  ax1.plot(
  epochs_seen, val_values, linestyle="-.",
      label=f"Validation {label}"
  )
  ax1.set_xlabel("Epochs")
  ax1.set_ylabel(label.capitalize())
  ax1.legend()

  ax2 = ax1.twiny()
  ax2.plot(examples_seen, train_values, alpha=0)
  ax2.set_xlabel("Examples seen")

  fig.tight_layout()
  plt.savefig(f"{label}-plot.pdf")
  plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))
plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

In [ ]:
import sys
import torch
from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_state = torch.load('/content/drive/MyDrive/gpt2_finetuned_sst.pt',
                         map_location=device)

!git clone https://github.com/zodbot/llm_finetuning.git

# Change into repo directory
%cd llm_finetuning

from src.model import GPTModel
from src.config import GPT_CONFIGS
from src.utils import load_weights_into_gpt
import torch

# Get configuration
config = GPT_CONFIGS["gpt2-small (124M)"]

model = GPTModel(config)

# 2. Modify model architecture for classification (just like during training)
num_classes = 2
model.out_head = torch.nn.Linear(config["emb_dim"], num_classes)

# load the state dict
model.load_state_dict(loaded_state['model_state_dict'])


In [ ]:
def calc_accuracy(data_loader, model, device, num_batches=None):
    if len(data_loader) == 0:
        return float("nan")

    if num_batches is None:
        num_batches = len(data_loader)

    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for i, (input_batch, target_batch) in enumerate(data_loader):
            if i >= num_batches:
                break

            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)

            # Forward pass
            outputs = model(input_batch)
            outputs = outputs[:, -1, :]  # Get last token predictions
            _, predicted = torch.max(outputs, 1)

            # Calculate accuracy
            total += target_batch.size(0)
            correct += (predicted == target_batch).sum().item()

    accuracy = correct / total if total > 0 else 0
    return accuracy  # Make sure we return the accuracy!

train_accuracy = calc_accuracy(train_loader, model, device, 4)
val_accuracy = calc_accuracy(val_loader, model, device, 4)
test_accuracy = calc_accuracy(test_loader, model, device, 4)
print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
def text_to_token_ids(text, tokenizer, max_length=None, pad_token_id=50256):
    # Encode text to token IDs
    tokens = tokenizer.encode(text)

    # Convert to tensor and add batch dimension
    tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension

    # Handle max length if specified
    if max_length is not None:
        if tokens.size(1) > max_length:
            # Truncate if too long
            tokens = tokens[:, :max_length]
        elif tokens.size(1) < max_length:
            # Pad if too short
            padding = torch.full((1, max_length - tokens.size(1)),
                               pad_token_id,
                               dtype=torch.long)
            tokens = torch.cat([tokens, padding], dim=1)

    return tokens

# Example usage:
# text = "This is a great movie!"
# token_ids = text_to_token_ids(text, tokenizer, max_length=512)
# print(token_ids.shape)  # Should be [1, sequence_length]
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()
    with torch.no_grad():
        # Tokenize the input text
        tokens = text_to_token_ids(text, tokenizer).to(device)

        # Get model prediction
        logits = model(tokens)
        logits = logits[:, -1, :]  # Get last token's logits

        # Convert to probabilities
        probs = torch.softmax(logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1).item()
        confidence = probs[0, prediction].item()

        # Convert to sentiment
        sentiment = "Positive" if prediction == 1 else "Negative"

        return {
            "sentiment": sentiment,
            "confidence": confidence,
            "probabilities": {
                "negative": probs[0, 0].item(),
                "positive": probs[0, 1].item()
            }
        }

# Example usage:
test_reviews = [
    "This movie was absolutely fantastic!",
    "What a terrible waste of time.",
    "The acting was okay, but the plot was confusing."
]

for review in test_reviews:
    result = classify_review(review, model, tokenizer, device)
    print(f"\nReview: {review}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Confidence: {result['confidence']:.2%}")
    print(f"Probabilities: Positive: {result['probabilities']['positive']:.2%}, "
          f"Negative: {result['probabilities']['negative']:.2%}")